In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.random.set_seed(1)

In [14]:
xy = np.loadtxt('Data/data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

[[1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 1. 0. 1.]
 [0. 0. 1. ... 1. 0. 0.]
 ...
 [1. 0. 0. ... 1. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 1. 0. 0.]]


In [12]:
nb_classes = 7
#분류를 위해 라벨값을 one-hot encoding
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print(Y_one_hot)


tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 

In [13]:
print(x_data.shape, Y_one_hot.shape)

(101, 16) (101, 7)


In [21]:
W = tf.Variable(tf.random.normal((x_data.shape[1], nb_classes)), name ='weight')
b = tf.Variable(tf.random.normal((nb_classes, )), name = 'bias')
variables = [W, b]

In [44]:
def logit_fn(X):
    return tf.matmul(X, W) + b


def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))


def cost_fn(X, Y):
    logits = logit_fn(X)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf.stop_gradient([Y])))
    return cost



def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads

    
def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_pred = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return accuracy


In [45]:
def fit(X, Y, epochs=1001, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.05)
    
    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(grads_and_vars = zip(grads, variables))
        if (i == 0) | ((i+1) % 100 == 0):
            print('epochs: {} |Loss: {}| Accuracy: {}'.format(i+1, cost_fn(X, Y).numpy(), prediction(X, Y).numpy()))
            


In [46]:
fit(x_data, Y_one_hot)

epochs: 1 |Loss: 5.272993564605713| Accuracy: 0.029702970758080482
epochs: 100 |Loss: 1.1562901735305786| Accuracy: 0.7128713130950928
epochs: 200 |Loss: 0.7251811027526855| Accuracy: 0.8316831588745117
epochs: 300 |Loss: 0.5664744973182678| Accuracy: 0.8712871074676514
epochs: 400 |Loss: 0.46894100308418274| Accuracy: 0.8811880946159363
epochs: 500 |Loss: 0.39836838841438293| Accuracy: 0.9009901285171509
epochs: 600 |Loss: 0.3439432978630066| Accuracy: 0.9009901285171509
epochs: 700 |Loss: 0.30092906951904297| Accuracy: 0.9009901285171509
epochs: 800 |Loss: 0.26639363169670105| Accuracy: 0.9009901285171509
epochs: 900 |Loss: 0.2382412701845169| Accuracy: 0.9207921028137207
epochs: 1000 |Loss: 0.21497339010238647| Accuracy: 0.9405940771102905


In [48]:
sample = [[1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 4, 0, 0, 1]]
sample = np.asarray(sample, dtype = np.float32)

In [49]:
a = hypothesis(sample)
print(a)

tf.Tensor(
[[9.9287790e-01 4.2199658e-04 2.4495916e-03 8.2124353e-07 1.1463002e-03
  1.8414221e-03 1.2619612e-03]], shape=(1, 7), dtype=float32)
